In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7" 

In [3]:
from sklearn import metrics
import numpy as np
import pandas as pd
from pathlib import Path
from typing import *
from sklearn.model_selection import train_test_split, StratifiedKFold

import torch
import torch.optim as optim
from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil
device=torch.device('cuda')

package_dir_a = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
sys.path.insert(0, package_dir_a)


from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from pytorch_pretrained_bert import OpenAIAdam

import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="1,2"  # specify which GPU(s) to be used

MAX_SEQUENCE_LENGTH = 220
SEED = 1234
EPOCHS = 1
Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification"
Input_dir = "../input"

#! mkdir ../working2
WORK_DIR = "../working2/"

TOXICITY_COLUMN = 'target'

#BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_L-12_H-768_A-12/'
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_L-24_H-1024_A-16/'
#shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

from pytorch_pretrained_bert import BertConfig

bert_config = BertConfig('../input/bert-pretrained-models/uncased_L-24_H-1024_A-16/'+'bert_config.json')

def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_L-24_H-1024_A-16/'

convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
    BERT_MODEL_PATH + 'bert_config.json',
    WORK_DIR + 'pytorch_model.bin')
shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'config.json')


/home/windywinter/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/windywinter/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/home/windywinter/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:4900: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/windywinter/.keras/keras.json' mode='r' encoding='UTF-8'>
  _config = json.load(open(_config_path))
/home/windywinter/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/windywinter/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversio

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /home/windywinter/JigSaw/input/bert-pretrained-models/uncased_L-24_H-1024_A-16/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [1024]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [1024]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 1024]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 1024]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 1024]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encode

Loading TF weight bert/encoder/layer_16/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_16/attention/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_16/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_16/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_16/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_16/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_16/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_16/attention/self/query/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_16/attention/self/value/bias with shape [1024]
Loading TF weight bert/encoder/layer_16/attention/self/value/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_16/intermediate/dense/bias with shape [4096]
Loading TF weight bert/encoder/la

Loading TF weight bert/encoder/layer_21/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_21/attention/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_21/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_21/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_21/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_21/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_21/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_21/attention/self/query/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_21/attention/self/value/bias with shape [1024]
Loading TF weight bert/encoder/layer_21/attention/self/value/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_21/intermediate/dense/bias with shape [4096]
Loading TF weight bert/encoder/la

Loading TF weight bert/encoder/layer_6/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_6/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_6/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/self/query/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_6/attention/self/value/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/self/value/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_6/intermediate/dense/bias with shape [4096]
Loading TF weight bert/encoder/layer_6/inter

'../working2/config.json'

In [3]:
#! cd bert_large_models && ls -lh

total 47G
-rw-rw-r-- 1 windywinter windywinter 1.3G Jun 23 04:30 bert_large_fold_0_epoch_1_ckpt_iteration_0.bin
-rw-rw-r-- 1 windywinter windywinter 1.3G Jun 23 11:49 bert_large_fold_0_epoch_1_ckpt_iteration_10000.bin
-rw-rw-r-- 1 windywinter windywinter 1.3G Jun 23 05:14 bert_large_fold_0_epoch_1_ckpt_iteration_1000.bin
-rw-rw-r-- 1 windywinter windywinter 1.3G Jun 23 12:34 bert_large_fold_0_epoch_1_ckpt_iteration_11000.bin
-rw-rw-r-- 1 windywinter windywinter 1.3G Jun 23 13:19 bert_large_fold_0_epoch_1_ckpt_iteration_12000.bin
-rw-rw-r-- 1 windywinter windywinter 1.3G Jun 23 14:04 bert_large_fold_0_epoch_1_ckpt_iteration_13000.bin
-rw-rw-r-- 1 windywinter windywinter 1.3G Jun 23 14:49 bert_large_fold_0_epoch_1_ckpt_iteration_14000.bin
-rw-rw-r-- 1 windywinter windywinter 1.3G Jun 23 15:34 bert_large_fold_0_epoch_1_ckpt_iteration_15000.bin
-rw-rw-r-- 1 windywinter windywinter 1.3G Jun 23 05:57 bert_large_fold_0_epoch_1_ckpt_iteration_2000.bin
-rw-rw-r-- 1 windywinter windywinter 1.3G 

In [8]:
#model = BertForSequenceClassification.from_pretrained("../working2", cache_dir=None, num_labels=7)

#model.load_state_dict(torch.load('{}/bert_large_fold_{}_lr_{}_STAGE_{}.bin'.format(OUTDIR, I, LR, 4)))


In [4]:
device=torch.device('cuda')
torch.cuda.device_count()
torch.cuda.get_device_name(0)
torch.cuda.get_device_properties(device).total_memory
torch.cuda.is_available()
torch.cuda.memory_allocated(device=None)
print('remaining cuda memory is: {}'.format(torch.cuda.get_device_properties(device).total_memory - torch.cuda.memory_allocated(device=None) ))

torch.cuda.empty_cache()

1

'Tesla V100-PCIE-16GB'

16945512448

True

0

remaining cuda memory is: 16945512448


In [5]:
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)

SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)


In [6]:
import pickle

train_df = pd.read_csv(os.path.join(Data_dir,"train.csv"))
print('loaded %d records' % len(train_df))
train_df['comment_text'] = train_df['comment_text'].astype(str) 
train_df=train_df.fillna(0)
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target']
train_df = train_df.drop(['comment_text'],axis=1)
train_df['target']=(train_df['target']>=0.5).astype(float)

with open('skf_5_splits.pkl', 'rb') as f:
    splits = pickle.load(f)

#skf = StratifiedKFold(n_splits=5, random_state=True, shuffle=True)
#splits = list(skf.split(train_df, train_df['target']))

lr=2e-5
I = 0
WARMUP=0.05
SEED = 1234
EPOCHS = 1

#sequences = np.load('jin_files/bert_base_sequences_220.npy')
sequences = np.load('jin_files/bert_large_sequences_220.npy')

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None, do_lower_case=True)

X = sequences[splits[I][0]]
X_val = sequences[splits[I][1]]
test_df = train_df.iloc[splits[I][1],:]

weights = np.ones(len(train_df)) / 4
weights += (train_df[identity_columns].fillna(0).values >= 0.5).sum(axis=1).astype(bool).astype(np.int) / 4
weights += (( (train_df['target'].values >= 0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values < 0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
weights += (( (train_df['target'].values < 0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values >= 0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4

Y = np.vstack([(train_df['target'].values >= 0.5).astype(np.int), weights]).T
Y_aux = train_df[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
Y = np.hstack((Y, Y_aux))

y = Y[splits[I][0]]
y_val = Y[splits[I][1]]

del train_df, sequences, Y, Y_aux, weights, splits
gc.collect()

train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))
del X, y, tokenizer
gc.collect()

def custom_loss(y_pred, y_true):
    bce_loss_1 = F.binary_cross_entropy_with_logits(y_pred[:,:1].reshape(-1), y_true[:,:1].reshape(-1), weight=y_true[:,1:2].reshape(-1))   
    N = y_true.shape[1]
    custom_loss = bce_loss_1
    for i in range(N-2):
        custom_loss += F.binary_cross_entropy_with_logits(y_pred[:,1+i], y_true[:,2+i])
    return custom_loss

batch_size = 32
accumulation_steps=1


loaded 1804874 records


0

0

In [7]:
## actual test size is around 0.97e-5
N = 30000
#N = 360975
# original size 360975
X_val_1 = X_val[:N,:]
test_df_1 = test_df.iloc[:N,:]


In [14]:
import torch.nn as nn

model = BertForSequenceClassification.from_pretrained("../working2", cache_dir=None, num_labels=7)  

#output_model_file = 'bert_large_models/bert_large_fold_0_epoch_1_ckpt_iteration_7000.bin'
#output_model_file = 'bert_large_models/bert_large_fold_0_lr_2e-05_STAGE_1.bin'
#output_model_file = 'bert_large_models/bert_large_fold_0_epoch_1_ckpt_iteration_13000.bin'    # full eval, 0.9398
#output_model_file = 'bert_large_models/bert_large_fold_0_lr_2e-05_STAGE_2.bin'                # .., 0.9397
#output_model_file = 'bert_large_models/bert_large_fold_0_epoch_2_ckpt_iteration_9000.bin'     # 0.9405669107668508
#output_model_file = 'bert_large_models/bert_large_fold_0_epoch_2_ckpt_iteration_11000.bin'    # 0.9413588399955286
#output_model_file = 'bert_large_models/bert_large_fold_0_epoch_2_ckpt_iteration_13000.bin'
#output_model_file = 'bert_large_models/bert_large_fold_0_lr_2e-05_STAGE_4.bin'
#output_model_file = 'bert_large_models/bert_large_fold_0_epoch_2_ckpt_iteration_14000.bin'

output_model_file = 'bert_large_models/bert_large_fold_0_lr_2e-05_STAGE_4.bin'
#output_model_file = 'bert_large_models/bert_large_full_epoch_2_ckpt_iteration_18000.bin'

model.load_state_dict(torch.load(output_model_file))


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [15]:
#output_model_file = '../working2/pytorch_model.bin'

#output_model_file = 'jin_files/bert_large_fold_2_STAGE_1.bin'
bs = 256

#model = BertForSequenceClassification(bert_config, num_labels=7)
#model.load_state_dict(torch.load(output_model_file))
model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()
valid_preds = np.zeros((len(X_val_1)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val_1,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=bs, shuffle=False)

tk0 = tqdm_notebook(valid_loader)


for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    #pred = model(x_batch.to(device), labels=None)        # gpt2 only
    valid_preds[i*bs:(i+1)*bs]=pred[:,0].detach().cpu().squeeze().numpy()
    

MODEL_NAME = 'model1'
test_df_1[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): FusedLayerNorm(torch.Size([1024]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([1024]), eps

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): FusedLayerNorm(torch.Size([1024]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([1024]), eps

A Jupyter Widget

/home/windywinter/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
from sklearn import metrics
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df_1, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
print(get_final_metric(bias_metrics_df, calculate_overall_auc(test_df_1, MODEL_NAME)))

,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
6,0.965087,0.895634,black,0.836693,241
5,0.964238,0.913892,muslim,0.863020,173
7,0.969577,0.909358,white,0.875432,285
0,0.969173,0.952582,male,0.935604,599
3,0.975469,0.942295,christian,0.938379,225
2,0.980655,0.916599,homosexual_gay_or_lesbian,0.941023,126
1,0.973881,0.954931,female,0.949210,689
8,0.981860,0.946921,psychiatric_or_mental_illness,0.961019,100
4,0.974303,0.973299,jewish,0.984375,48


0.9484513944662412


In [ ]:
# STAGE 3: 0.9470025559495141

# full 4: 0.9556051496446678

In [18]:
from sklearn import metrics
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df_1, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
print(get_final_metric(bias_metrics_df, calculate_overall_auc(test_df_1, MODEL_NAME)))

,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
5,0.957011,0.918530,muslim,0.816667,22
6,0.969006,0.925476,black,0.869186,59
2,0.973629,0.927555,homosexual_gay_or_lesbian,0.906760,50
7,0.979628,0.935564,white,0.937500,96
0,0.969556,0.963775,male,0.940952,196
1,0.971351,0.967060,female,0.946393,241
8,0.979676,0.961070,psychiatric_or_mental_illness,0.950758,35
3,0.979587,0.962650,christian,0.962500,95
4,0.986559,0.962118,jewish,0.982456,22


0.9538083010177182


# CV validation for Bert Large 220

In [11]:
from sklearn import metrics
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df_1, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
print(get_final_metric(bias_metrics_df, calculate_overall_auc(test_df_1, MODEL_NAME)))
#test_df_1.to_csv('bert_large_STAGE_4_test_df.csv')

,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
2,0.961273,0.900299,homosexual_gay_or_lesbian,0.846619,2057
6,0.969783,0.888902,black,0.868668,2920
7,0.967850,0.903271,white,0.876645,4875
5,0.964331,0.921852,muslim,0.892012,3935
4,0.954105,0.946253,jewish,0.903139,1462
1,0.962050,0.956896,female,0.936402,10127
0,0.966795,0.951262,male,0.938215,8057
3,0.953661,0.965647,christian,0.938589,7065
8,0.975468,0.940663,psychiatric_or_mental_illness,0.946386,836


0.9414556767943845


In [19]:
from sklearn import metrics
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df_1, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
print(get_final_metric(bias_metrics_df, calculate_overall_auc(test_df_1, MODEL_NAME)))


,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
2,0.958345,0.907456,homosexual_gay_or_lesbian,0.847023,2057
6,0.969497,0.889002,black,0.868132,2920
7,0.968384,0.900635,white,0.876086,4875
5,0.964730,0.919856,muslim,0.891030,3935
4,0.954909,0.944742,jewish,0.902340,1462
1,0.961301,0.957684,female,0.936479,10127
0,0.965941,0.951889,male,0.937689,8057
3,0.953795,0.965578,christian,0.938905,7065
8,0.975496,0.940641,psychiatric_or_mental_illness,0.947146,836


0.9413588399955286


In [11]:
#test_df_1.to_csv('bert_large_it_9000_test_df.csv')
#test_df_1.to_csv('bert_large_it_11000_test_df.csv')
test_df_1.to_csv('bert_large_it_13000_test_df.csv')

In [16]:
from sklearn import metrics
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df_1, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
print(get_final_metric(bias_metrics_df, calculate_overall_auc(test_df_1, MODEL_NAME)))



,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
6,0.964067,0.878860,black,0.827035,59
5,0.962007,0.882886,muslim,0.833333,22
2,0.975833,0.871489,homosexual_gay_or_lesbian,0.892774,50
8,0.944134,0.944030,psychiatric_or_mental_illness,0.901515,35
7,0.975192,0.887770,white,0.901974,96
0,0.958766,0.946502,male,0.920000,196
1,0.970638,0.944893,female,0.939925,241
3,0.974875,0.938401,christian,0.954167,95
4,0.981111,0.943953,jewish,0.982456,22


0.9367616556216991


In [27]:
from sklearn import metrics
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df_1, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
print(get_final_metric(bias_metrics_df, calculate_overall_auc(test_df_1, MODEL_NAME)))


,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
8,0.889027,0.713272,psychiatric_or_mental_illness,0.652287,100
7,0.899404,0.659274,white,0.669015,285
6,0.930414,0.613429,black,0.677137,241
5,0.929013,0.637829,muslim,0.759476,173
2,0.966795,0.595024,homosexual_gay_or_lesbian,0.811665,126
3,0.925078,0.777396,christian,0.826992,225
0,0.917040,0.799276,male,0.839162,599
1,0.943194,0.779749,female,0.871151,689
4,0.932647,0.845854,jewish,0.903125,48


0.8107619830385726


In [28]:
torch.cuda.device_count()

8

In [ ]:
from sklearn.metrics import roc_auc_score
power_mean(bias_metrics_df['subgroup_auc'], p=-5)
power_mean(bias_metrics_df['bpsn_auc'], p=-5)
power_mean(bias_metrics_df['bnsp_auc'], p=-5)
print(roc_auc_score(test_df['target'], test_df['model1']))

In [7]:
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
print(get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME)))


,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
2,0.939463,0.621428,homosexual_gay_or_lesbian,0.698807,2022
6,0.927327,0.658027,black,0.699959,2736
7,0.930065,0.654232,white,0.717301,4744
5,0.907243,0.735618,muslim,0.739049,3847
8,0.859440,0.848040,psychiatric_or_mental_illness,0.794291,847
4,0.910647,0.781070,jewish,0.797108,1447
0,0.930022,0.798668,male,0.848262,7975
1,0.924740,0.815904,female,0.857535,10079
3,0.914551,0.846384,christian,0.865211,7204


0.8241944469617198


In [ ]:
test_df